[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/reznikmm/ada-howto/advent-2020?filepath=%2Fhome%2Fjovyan%2Fnb%2F16%2F16.ipynb)

# Day 16
Firstly, let's force assertion checking. We will use `pragma Assert` in the code.

In [1]:
pragma Assertion_Policy (Check);

We need some standard packages to read inputs and keep a program state.

In [2]:
with Ada.Text_IO;
with Ada.Integer_Text_IO;
with Ada.Strings.Fixed;
with Ada.Strings.Maps;
with Ada.Strings.Maps.Constants;

A field validation rule has two ranges.

In [3]:
type Valid_Range is record
   From, To : Positive;
end record;

type Field_Validation is array (1 .. 2) of Valid_Range;

A ticket validation has such field validation for eash of 20 fields. Let's define it as an array type. For index type we use a dedicated integer type. It helps us not to mix it with oher indexes and integers.

In [4]:
type Field_Index is range 1 .. 20;  --  As they go in rules

type Ticket_Validation is array (Field_Index) of Field_Validation;

Let's define a tool subprogram to find an integer in the given string.

In [5]:
procedure Find_Decimal
  (Line : String;
   From : out Positive;
   To   : in out Natural) is
begin
   Ada.Strings.Fixed.Find_Token
     (Source => Line,
      Set    => Ada.Strings.Maps.Constants.Decimal_Digit_Set,
      From   => To + 1,
      Test   => Ada.Strings.Inside,
      First  => From,
      Last   => To);
end Find_Decimal;

The `Input` file to read the input data.

In [6]:
Input : Ada.Text_IO.File_Type;

A procedure to read line from `Input` and decode field validation ranges.

In [7]:
procedure Read_Field_Validation (Result : out Field_Validation) is
   Line : String := Ada.Text_IO.Get_Line (Input);
   First, Last : Positive := 1;
begin
   for J of Result loop
      Find_Decimal (Line, First, Last);
      J.From := Positive'Value (Line (First .. Last));
      Find_Decimal (Line, First, Last);
      J.To := Positive'Value (Line (First .. Last));
   end loop;
end Read_Field_Validation;

Another procedure to read a batch of filed validation ranges:

In [8]:
procedure Read_Validation (Result : out Ticket_Validation) is
begin
   for Field of Result loop
      Read_Field_Validation (Field);
   end loop;

   pragma Assert (Ada.Text_IO.Get_Line (Input) = "");
end Read_Validation;

Now we can read the validation rules:

In [9]:
Rules : Ticket_Validation;

Let's open a file and start reading:

In [10]:
Ada.Text_IO.Open (Input, Ada.Text_IO.In_File, "/home/jovyan/md/14/input");

Read_Validation (Rules);

Let's define a ticket as array of integer components.

In [11]:
type Ticket is array (1 .. 20) of Positive;

And a procedure to read a ticket data from `Input`.

In [12]:
procedure Read_Ticket (Value : out Ticket) is
   Comma : Character;
begin
   for J in Value'Range loop
      if J /= 1 then
         Ada.Text_IO.Get (Input, Comma);
      end if;

      Ada.Integer_Text_IO.Get (Input, Value (J));
   end loop;
   pragma Assert (Ada.Text_IO.Get_Line (Input) = "");
end Read_Ticket;

Keep my ticket data in `My_Ticket`.

In [13]:
My_Ticket : Ticket;

Let's read it

In [14]:
pragma Assert (Ada.Text_IO.Get_Line (Input) = "");
pragma Assert (Ada.Text_IO.Get_Line (Input) = "your ticket:");
Read_Ticket (My_Ticket);
pragma Assert (Ada.Text_IO.Get_Line (Input) = "");

Another simple function will check if ticket data is corrent.

In [15]:
function Check_Ticket
  (Value : Ticket;
   Rules : Ticket_Validation) return Boolean;
--  Verify if given all fields in ticket match some range in rules

function Check_Ticket
  (Value : Ticket;
   Rules : Ticket_Validation) return Boolean
is
   Result : Boolean := True;
begin
   for Field of Value loop
      declare
         Found : Boolean := False;
      begin
         for Rule of Rules loop
            Found := Found
              or Field in Rule (1).From .. Rule (1).To
              or Field in Rule (2).From .. Rule (2).To;
         end loop;

         if not Found then
            Result := False;
         end if;
      end;
   end loop;

   return Result;
end Check_Ticket;

Let's collect for each ticket fields a set of valid rules. Set `True` if a rule suits the field and `False` otherwise.

In [16]:
type Field_Mask is array (Field_Index) of Boolean;

type Field_Masks is array (1 .. 20) of Field_Mask;

Masks : Field_Masks := (others => (others => True));

Procedure `Update_Field_Masks` looks like the `Check_Field` but it modifies masks.

In [17]:
procedure Update_Field_Masks
  (Value : Ticket;
   Rules : Ticket_Validation;
   Masks : in out Field_Masks);
--  Clear flags in Masks for fields of a ticket that unmatch rules.

procedure Update_Field_Masks
  (Value : Ticket;
   Rules : Ticket_Validation;
   Masks : in out Field_Masks) is
begin
   for F in Value'Range loop
      for R in Rules'Range loop
         if Value (F) not in
           Rules (R) (1).From .. Rules (R) (1).To |
           Rules (R) (2).From .. Rules (R) (2).To
         then
            Masks (F) (R) := False;
         end if;
      end loop;
   end loop;
end Update_Field_Masks;

Now read nearby tickets, check them and fix `Masks`.

In [18]:
pragma Assert (Ada.Text_IO.Get_Line (Input) = "");
pragma Assert (Ada.Text_IO.Get_Line (Input) = "nearby tickets:");

while not Ada.Text_IO.End_Of_File (Input) loop
   declare
      Value : Ticket;
   begin
      Read_Ticket (Value);
      if Check_Ticket (Value, Rules) then
         Update_Field_Masks (Value, Rules, Masks);
      end if;
   end;
end loop;

Now let's fime a mapping between field descriptions and actual fields.

In [19]:
type Index_Mapping is array (Field_Index) of Natural;

Let's find a row in `Masks` where only one `True` flag. Then fill corresponding component if `Mapping` and clear this flag in all rows. Do this 20 times.

In [20]:
procedure Find_Mapping
  (Masks   : in out Field_Masks;
   Mapping : out Index_Mapping);
--  Populate Mapping according to Masks

procedure Find_Mapping
  (Masks   : in out Field_Masks;
   Mapping : out Index_Mapping)
is
   procedure Find_Unique (Field : out Field_Index; Index : out Positive) is
   begin
      for J in Masks'Range loop
         declare
            Found : Boolean := False;
         begin
            for K in Masks (J)'Range loop
               if Masks (J) (K) then
                  if Found then
                     Found := False;
                     exit;
                  else
                     Found := True;
                     Field := K;
                     Index := J;
                  end if;
               end if;
            end loop;

            if Found then
               return;
            end if;
         end;
      end loop;

      raise Program_Error;
   end Find_Unique;

begin
   for J in 1 .. 20 loop
      declare
         Field : Field_Index;
         Index : Positive;
      begin
         Find_Unique (Field, Index);
         Mapping (Field) := Index;

         declare
            Mask : Field_Mask := (others => True);
         begin
            Mask (Field) := False;
            for Item of Masks loop
               Item := Item and Mask;
            end loop;
         end;
      end;
   end loop;
end Find_Mapping;

Now we are ready to find result:

In [21]:
declare
   Mapping : Index_Mapping := (others => 0);
   Result : Long_Integer := 1;
begin
   Find_Mapping (Masks, Mapping);
   for J in Field_Index'(1) .. 6 loop
      Result := Result * Long_Integer (My_Ticket (Mapping (J)));
   end loop;
   Ada.Text_IO.Put_Line (Result'Image);
end;

 2628667251989


----
[Back to Table of Contents](https://github.com/reznikmm/ada-howto/tree/advent-2020) 